In [1]:
import seismometer as sm
sm.run_startup(config_path='.')

In [ ]:
sm.show_info()

# write up
* config updates
  * usage - metrics section + config; group key; count vs prevalence
  * data dictionary - categorical values display name
* analysis
  * ordinal categorical graph - include N/A handling in usage
* explore controls
  * group key

todos -
attach kwargs to super

In [3]:
from seismometer.controls.explore import ExplorationSubpopulationWidget
from seismometer.plot.mpl.decorators import render_as_svg
import matplotlib.pyplot as plt
import plot_likert
from typing import Any, Callable, Optional

mkwargs = {}

class OrdinalCategoricalWidget(ExplorationSubpopulationWidget):
    """
    A widget for exploring the model performance of a cohort.
    """

    def __init__(
        self,
        **kwargs
    ):
        """
        Exploration widget for model evaluation, showing a plot for a given target,
        score, threshold, and cohort selection.

        Parameters
        ----------
        title : str
            title of the control
        plot_function : Callable[..., Any]
            Expected to have the following signature:

            .. code:: python

                def plot_function(
                    cohorts: dict[str,tuple[Any]]) -> Any
        """
        super().__init__(
            title="Rubric Metrics",
            plot_function=ordinal_plot,
            **kwargs
        )

@render_as_svg
def ordinal_plot(
            cohort_dict: str) -> plt.Figure:
    from seismometer.seismogram import Seismogram

    from seismometer.data.filter import filter_rule_from_cohort_dictionary
    from seismometer.data import pandas_helpers as pdh

    sg = Seismogram()
    cfg = sg.config

    '''
    if merge_strategy not in get_args(MergeStrategies):
        raise ValueError(
            f"Invalid merge strategy {merge_strategy} for {event_label}."
            + f" Must be one of: {', '.join(get_args(MergeStrategies))}."
        )
    '''

    group_key = self.kwargs.get('group_key', None)

    cols = cfg.get_metrics_by_type('ordinal_categorical', group_key)

    rule = filter_rule_from_cohort_dictionary(cohort_dict)
    data = rule.filter(sg.dataframe)[
        cfg.entity_keys + cols #
    ]


    data.to_csv('data.csv')

    cohort_count = data[sg.entity_keys[0]].nunique()
    if cohort_count < sg.censor_threshold:
        return 'not enough data' #template.render_censored_plot_message(sg.censor_threshold)



    tf = data[cols]

    p = plot_likert.plot_likert(tf, plot_percentage=True, plot_scale=plot_likert.scales.agree)

    return p

In [ ]:
OrdinalCategoricalWidget(group_key="Rubric metrics")

In [5]:
from seismometer.seismogram import Seismogram

In [6]:
sg = Seismogram()
cfg = sg.config

In [ ]:
cfg.usage

In [ ]:
cfg.get_metrics_by_type('ordinal_categorical')

In [ ]:
cfg.get_metrics_by_type('ordinal_categorical', 'User feedback')

In [ ]:
cfg.get_metrics_by_type('ordinal_categorical', 'Rubric metrics')